In [25]:
import os
import torch
from PIL import Image
from torchvision import transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader, random_split

In [26]:
class FacesDataset(Dataset):
    def __init__(self, directory, transform=None):
        self.directory = directory
        self.transform = transform
        self.labels, self.images = self.load_images(directory)

    def load_images(self, directory):
        labels = []
        images = []
        for label in os.listdir(directory):
            label_path = os.path.join(directory, label)
            for image in os.listdir(label_path):
                images.append(os.path.join(label_path, image))
                labels.append(label)
        return labels, images

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx])
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [27]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [28]:
directory = r'C:\Users\adam.bloebaum\.vscode\driveline\cv\data\images'
dataset = FacesDataset(directory, transform=transform)

In [29]:
# Splitting the dataset
total_size = len(dataset)
train_size = int(total_size * 0.8)
val_size = total_size - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Creating data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

In [30]:
def count_classes(directory):
    """Count the number of subdirectories in a given directory."""
    return len([name for name in os.listdir(directory) 
                if os.path.isdir(os.path.join(directory, name))])

num_classes = count_classes(directory)
print(f"Number of classes (athletes): {num_classes}")

Number of classes (athletes): 2


In [ ]:
# Using a pre-trained model, for example, ResNet
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, num_classes)  # num_classes is the number of athletes

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Learning rate may need tuning

# Check if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
# Training Loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")

    # Validation phase
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Validation Accuracy: {100 * correct / total}%')

print("Training Complete")

In [ ]:
torch.save(model.state_dict(), 'best_model.pth')